# Interactive Plotting of Open Flight Data with Datashader

## TODO
* Plot full day of data
* Try with short trail days
* Use new agg files with categories and use for coloring

In [1]:
import os
import bokeh
import bokeh.plotting as plotting
from bokeh.models import WMTSTileSource
from bokeh.palettes import Category20, Category20b
plotting.output_notebook()
import datashader.transfer_functions as tf
import datashader as ds
from datashader.colors import viridis
from datashader.bokeh_ext import create_ramp_legend, create_categorical_legend
from bokeh.io import show
from datashader.bokeh_ext import InteractiveImage
from datashader.utils import lnglat_to_meters as webm
import dask
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
ProgressBar().register()
import pandas as pd
import numpy as np
from datashader.bokeh_ext import create_categorical_legend
np.warnings.filterwarnings('ignore')

Loading BokehJS ...

In [2]:
print(f'Pandas version: {pd.__version__}')
print(f'Bokeh version: {bokeh.__version__}')
print(f'Datashader version: {ds.__version__}')
print(f'dask version: {dask.__version__}')

Pandas version: 0.23.4
Bokeh version: 0.13.0
Datashader version: 0.6.8
dask version: 0.17.4


In [3]:
import datetime as dt
import logging
now = dt.datetime.now().strftime('%m-%d-%y_%H%M%S')
logging.basicConfig(filename=now+".log", level=logging.INFO, format='%(asctime)s %(message)s')

In [4]:
%%time
main_ops =  ['Southwest', 'American', 'Delta', 'SkyWest', 'Air Canada', 'Alaska', 
        'Virgin', 'United','JetBlue', 'Spirit', 'Frontier', 'Wells Fargo', 
        'WestJet','Private','British Airways', 'Aeroflot','Republic','Qantas',
        'Air France','Lufthansa','Jetstar','Wizz','Compass', 'Aeroméxico']

row_len = 500_000
h5_dir = r'c:\adsb'
h5_file = os.path.join(h5_dir, '2018-01-02-ST.h5')
pickle_name = f'{os.path.basename(h5_file)}-{row_len}.p'
pickle_path = os.path.join(os.getcwd(), 'data', pickle_name)
if os.path.exists(pickle_path):
    df = pd.read_pickle(pickle_path)
else:
    with pd.HDFStore(h5_file, mode='r') as store:
        df = store.select('data', stop = row_len, columns=['Lat', 'Long','Op'])


    for o in main_ops:
        df.loc[df.Op.fillna('Other').str.lower().str.contains(o.lower()), 'Op'] = o
    df.loc[~df.Op.isin(main_ops),'Op'] = 'Other'
    df = df.astype({'Op':'category'})
    
    if 'x' not in df.columns:
        w = webm(df.Long, df.Lat)
        df.loc[:,'x'] = w[0]
        df.loc[:,'y'] = w[1]
        df = df.drop(columns=['Lat','Long'])
    df.to_pickle(pickle_path)

Wall time: 32.9 ms


In [5]:
color_mapper26 = Category20[20] + Category20b[20][2::3]
color_mapper_op = {v:c for v,c in zip(main_ops + ['Other'], color_mapper26)}

In [6]:
f'Row Count: {len(df):,}'

'Row Count: 500,000'

In [7]:
MaxBounds = ((-20048966.10, 20048966.10), (-20026376.39, 20026376.39))
WholeWorld = ((-20_037_508, 20_037_508), (-7_670_608, 13_971_466))
TwoBounds = ((-20_000_000, 20_000_000), (-20_000_000, 20_000_000))
USA_CONUS = ((-13884029, -7453304), (2698291, 6455972))
WesternEuro = ((-1181114, 4270391), (3000000, 8081620))
Germany = ((709336, 1600000), (6026907, 7270000))
Chicago = (( -9828281, -9717659), (5096658, 5161298))
Chinatown = (( -9759210, -9754583), (5137122, 5139825))
NewYorkCity = (( -8280656, -8175066), (4940514, 4998954))
LosAngeles = ((-13195052, -13114944), (3979242, 4023720))
Houston = ((-10692703, -10539441), (3432521, 3517616))
Austin = ((-10898752, -10855820), (3525750, 3550837))
NewOrleans = ((-10059963, -10006348), (3480787, 3510555))
Atlanta = ((-9507853,-9274873), (3927030, 4069506))

In [8]:
# source: https://leaflet-extras.github.io/leaflet-providers/preview/
Esri_NatGeoWorldMap = 'https://server.arcgisonline.com/ArcGIS/rest/services/NatGeo_World_Map/MapServer/tile/{z}/{y}/{x}'
Esri_OceanBasemap = 'https://server.arcgisonline.com/ArcGIS/rest/services/Ocean_Basemap/MapServer/tile/{z}/{y}/{x}'
CartoDB_Positron = 'https://cartodb-basemaps-{s}.global.ssl.fastly.net/light_all/{z}/{x}/{y}{r}.png'
CartoDB_Voyager = 'https://cartodb-basemaps-{s}.global.ssl.fastly.net/rastertiles/voyager/{z}/{x}/{y}{r}.png'
OpenStreetMap_Mapnik = 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png'
OpenTopoMap = 'https://{s}.tile.opentopomap.org/{z}/{x}/{y}.png'
Hydda_Full = 'https://{s}.tile.openstreetmap.se/hydda/full/{z}/{x}/{y}.png'
Esri_WorldStreetMap = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{z}/{y}/{x}'
Esri_WorldTopoMap = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/{z}/{y}/{x}'
Esri_WorldImagery = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}'

In [9]:
def spread(pts):
    return ((pts[0][1] - pts[0][0]),
            (pts[1][1] - pts[1][0]))
def ratio(pts):
    s = spread(pts)
    x, y = s
    return x / y

In [10]:
def base_plot(xrange, yrange, plot_width=int(850), plot_height=int(500),
              tools='pan,wheel_zoom,zoom_in,zoom_out,reset', 
              bok_cir = True, tile_url=CartoDB_Positron):
    p = plotting.figure(tools=tools,
                  plot_width=plot_width, plot_height=plot_height,
                  x_range=(xrange), y_range=yrange, outline_line_color=None,
                  min_border=0, min_border_left=0, min_border_right=0,
                  min_border_top=0, min_border_bottom=0)
    p.match_aspect = True
    p.axis.visible = False
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    if bok_cir:
        p.circle(x="x", y="y",color='red', size=2, alpha=0.4) 
    tile_renderer = p.add_tile(WMTSTileSource(url=tile_url)) 
    tile_renderer.alpha=0.8
    return p

In [11]:
def create_image(x_range, y_range, plot_width, plot_height):
    logging.info(f'Create Image - {x_range}, {y_range}')
    cmap=viridis
    color_key = color_mapper_op
    cvs = ds.Canvas(plot_width, plot_height, x_range, y_range)
    agg = cvs.points(df, 'x', 'y', ds.count_cat('Op'))
    img = tf.shade(agg, cmap = cmap, color_key=color_key)
    img = tf.dynspread(img, threshold=0.5, max_px=4)
    logging.info(f'Image Generated')
    return img

def image_callback(xr, yr, w, h):
    logging.info(f'Callback')
    return create_image(xr, yr, w, h)

In [12]:
p = base_plot(*WholeWorld, bok_cir=False, tile_url=Esri_NatGeoWorldMap)
logging.info(f'InteractiveImage Start')
InteractiveImage(p, image_callback)